In [27]:
#Importing libraries
import yfinance as yf
import talib
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import sklearn
import xgboost
import dateutil
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from dateutil.relativedelta import relativedelta
from scipy.stats import linregress


from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import f1_score

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)


# Data Preparation

In [2]:
# I chose Intel asset to predict open and close
focus_asset=["INTC"]
# And then I added sector assets data 
peer_data_assets=["AMD", "NVDA", "QCOM","TXN","MU","AVGO","SSNLF","TSM","MSFT","AAPL","NVDA","AVGO","ORCL","ADBE",
                "QCOM","CSCO","AVGO","NXPI","MU"]


In [3]:
# Then indexes
indexes_=["^GSPC",
"^DJI",
"^IXIC",
"^NYA",
"^XAX",
"^BUK100P",
"^RUT",
"^VIX",
"^FTSE",
"^GDAXI",
"^FCHI",
"^STOXX50E",
"^N100",
"^BFX",
"IMOEX.ME",
"^N225",
"^HSI",
"000001.SS",
"399001.SZ",
"^STI",
"^AXJO",
"^AORD",
"^BSESN",
"^JKSE",
"^KLSE",
"^NZ50",
"^KS11",
"^TWII",
"^GSPTSE",
"^BVSP",
"^MXX",
"^MERV",
"^TA125.TA",
"^JN0U.JO"]


In [4]:
for e in indexes_:
    peer_data_assets.append(e)

In [5]:
START_DATE = '2022-01-01'
END_DATE = '2024-05-30' 
n_assets = len(focus_asset)

intel = yf.download(focus_asset, start=START_DATE,
                        end=END_DATE,interval="1d")
print(f'Downloaded {intel.shape[0]} rows of data.')

[*********************100%%**********************]  1 of 1 completed

Downloaded 604 rows of data.


In [6]:
n_assets = len(peer_data_assets)

peers = yf.download(peer_data_assets, start=START_DATE,
                        end=END_DATE,interval="1d")['Adj Close']
print(f'Downloaded {peers.shape[0]} rows of data.')

[*********************100%%**********************]  48 of 48 completed


Downloaded 736 rows of data.


#### I calculated returns of peers and focus assets

In [7]:
for i in peers.columns:
    peers["Return_"+i]=peers[i].pct_change()

In [8]:
peers

Ticker,000001.SS,399001.SZ,AAPL,ADBE,AMD,AVGO,CSCO,IMOEX.ME,MSFT,MU,NVDA,NXPI,ORCL,QCOM,SSNLF,TSM,TXN,^AORD,^AXJO,^BFX,^BSESN,^BUK100P,^BVSP,^DJI,^FCHI,^FTSE,^GDAXI,^GSPC,^GSPTSE,^HSI,^IXIC,^JKSE,^JN0U.JO,^KLSE,^KS11,^MERV,^MXX,^N100,^N225,^NYA,^NZ50,^RUT,^STI,^STOXX50E,^TA125.TA,^TWII,^VIX,^XAX,Return_000001.SS,Return_399001.SZ,Return_AAPL,Return_ADBE,Return_AMD,Return_AVGO,Return_CSCO,Return_IMOEX.ME,Return_MSFT,Return_MU,Return_NVDA,Return_NXPI,Return_ORCL,Return_QCOM,Return_SSNLF,Return_TSM,Return_TXN,Return_^AORD,Return_^AXJO,Return_^BFX,Return_^BSESN,Return_^BUK100P,Return_^BVSP,Return_^DJI,Return_^FCHI,Return_^FTSE,Return_^GDAXI,Return_^GSPC,Return_^GSPTSE,Return_^HSI,Return_^IXIC,Return_^JKSE,Return_^JN0U.JO,Return_^KLSE,Return_^KS11,Return_^MERV,Return_^MXX,Return_^N100,Return_^N225,Return_^NYA,Return_^NZ50,Return_^RUT,Return_^STI,Return_^STOXX50E,Return_^TA125.TA,Return_^TWII,Return_^VIX,Return_^XAX
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2073.090088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-03,NaN,NaN,179.273621,564.369995,150.240005,62.370541,57.983242,3852.500000,326.940796,94.188248,30.073059,219.249969,84.268791,175.001373,2.395472e+10,122.335548,175.891220,NaN,NaN,4335.279785,59183.218750,732.270020,103922.0,36585.058594,7217.220215,NaN,16020.730469,4796.560059,NaN,23274.750000,15832.799805,6665.308105,4532.049805,1549.050049,NaN,85579.0,52941.011719,1372.060059,NaN,17226.099609,NaN,2272.560059,3134.250000,4331.819824,2086.850098,18270.509766,16.600000,3461.719971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006637,NaN,NaN,NaN
2022-01-04,3632.330078,14791.309570,176.998352,554.000000,144.419998,63.085148,56.561138,3873.489990,321.334717,94.768616,29.243383,220.814590,85.169952,175.959976,2.395472e+10,126.704697,176.075790,7926.799805,7589.799805,4362.209961,59855.929688,743.950012,103514.0,36799.648438,7317.410156,7505.200195,16152.610352,4793.540039,21236.500000,23289.839844,15622.719727,6695.373047,4603.919922,1541.900024,2989.239990,85286.0,53016.781250,1383.489990,29301.789062,17336.759766,NaN,2268.870117,3181.129883,4367.620117,2088.530029,18526.349609,16.910000,3496.050049,NaN,NaN,-0.012692,-0.018374,-0.038738,0.011457,-0.024526,0.005448,-0.017147,0.006162,-0.027589,0.007136,0.010694,0.005478,0.0,0.035714,0.001049,NaN,NaN,0.006212,0.011367,0.015950,-0.003926,0.005866,0.013882,NaN,0.008232,-0.000630,NaN,0.000648,-0.013269,0.004511,0.015858,-0.004616,NaN,-0.003424,0.001431,0.008330,NaN,0.006424,NaN,-0.001624,0.014957,0.008264,0.000805,0.014003,0.018675,0.009917
2022-01-05,3595.179932,14525.759766,172.290207,514.429993,136.149994,60.459896,55.665394,3815.050049,308.999420,92.860268,27.560068,212.944275,82.888268,175.273880,2.395472e+10,120.682884,172.402908,7899.600098,7565.799805,4344.620117,60223.148438,746.140015,101006.0,36407.109375,7376.370117,7516.899902,16271.750000,4700.580078,21039.699219,22907.250000,15100.169922,6662.298828,4673.850098,1547.949951,2953.969971,83836.0,53024.148438,1388.089966,29332.160156,17112.599609,13150.379883,2194.000000,3163.439941,4392.149902,2105.479980,18499.960938,19.730000,3469.100098,-0.010228,-0.017953,-0.026600,-0.071426,-0.057264,-0.041614,-0.015837,-0.015087,-0.038388,-0.020137,-0.057562,-0.035642,-0.026790,-0.003899,0.0,-0.047526,-0.020860,-0.003431,-0.003162,-0.004032,0.006135,0.002944,-0.024229,-0.010667,0.008057,0.001559,0.007376,-0.019393,-0.009267,-0.016427,-0.033448,-0.004940,0.015189,0.003924,-0.011799,-0.017002,0.00

In [9]:
intel["Return"]=intel["Adj Close"].pct_change()

In [10]:
new_colnames=[]
for i in intel.columns:
    new_colnames.append("Intel_"+i)
intel.columns=new_colnames

In [11]:
intel

,Intel_Open,Intel_High,Intel_Low,Intel_Close,Intel_Adj Close,Intel_Volume,Intel_Return
Date,,,,,,,
2022-01-03,51.650002,53.230000,51.599998,53.209999,49.350483,33857600,NaN
2022-01-04,53.570000,53.939999,52.650002,53.139999,49.285557,45681400,-0.001316
2022-01-05,54.189999,56.169998,53.830002,53.869999,49.962608,59109300,0.013737
2022-01-06,54.610001,54.669998,53.419998,54.009998,50.092457,35757900,0.002599
2022-01-07,54.189999,54.389999,53.150002,53.439999,49.563808,30717200,-0.010553
...,...,...,...,...,...,...,...
2024-05-22,31.910000,32.070000,31.100000,31.420000,31.221941,36706400,-0.010082
2024-05-23,31.450001,31.570000,29.870001,30.080000,29.890387,62014500,-0.042648
2024-05-24,30.290001,31.020000,30.129999,30.719999,30.526352,42408200,0.021277


#### Then I calculated indicators of Intel asset by using Ta-Lib

In [12]:
intel['EMA_10'] = talib.EMA(intel['Intel_Close'], timeperiod=10)
intel['EMA_12'] = talib.EMA(intel['Intel_Close'], timeperiod=12)
intel['EMA_50'] = talib.EMA(intel['Intel_Close'], timeperiod=50)
intel['EMA_100'] = talib.EMA(intel['Intel_Close'], timeperiod=100)

intel["SMA_10"] = talib.SMA(intel['Intel_Close'], timeperiod=10)
intel['SMA_12'] = talib.SMA(intel['Intel_Close'], timeperiod=12)
intel['SMA_50'] = talib.SMA(intel['Intel_Close'], timeperiod=50)
intel['SMA_100'] = talib.SMA(intel['Intel_Close'], timeperiod=100)

intel['STD_10']=intel['Intel_Adj Close'].rolling(10).std()
intel['STD_12']=intel['Intel_Adj Close'].rolling(12).std()
intel['STD_50']=intel['Intel_Adj Close'].rolling(50).std()
intel['STD_100']=intel['Intel_Adj Close'].rolling(100).std()

intel['ADX_14'] = talib.ADX(intel['Intel_High'],intel['Intel_Low'],intel['Intel_Close'], timeperiod = 14)
intel["ADX_21"] = talib.ADX(intel['Intel_High'],intel['Intel_Low'],intel['Intel_Close'], timeperiod = 21)

intel['RSI_14'] = talib.RSI(intel['Intel_Close'], timeperiod = 14)
intel['RSI_21'] = talib.RSI(intel['Intel_Close'], timeperiod = 21)

intel["MACD"], intel["MACDSignal"], intel["MACDHist"] = talib.MACD(
intel['Intel_Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)

intel['UPPER_BBand_20'], intel['MID_BBand_20'],intel['LOWER_BBand_20'] = talib.BBANDS(intel['Intel_Close'],nbdevup=2,nbdevdn=2,timeperiod=20)

intel['Volatility'] = talib.TRANGE(intel['Intel_High'], intel['Intel_Low'], intel['Intel_Close'])

intel['Volatility'] = talib.TRANGE(intel['Intel_High'], intel['Intel_Low'], intel['Intel_Close'])
intel['STOCH_K'], intel['STOCH_D'] = talib.STOCH(intel['Intel_High'], intel['Intel_Low'], intel['Intel_Close'])


intel['CCI_14'] = talib.CCI(intel['Intel_High'], intel['Intel_Low'], intel['Intel_Close'],timeperiod=14)
intel['CCI_20'] = talib.CCI(intel['Intel_High'], intel['Intel_Low'], intel['Intel_Close'], timeperiod=20)

intel['OBV'] = talib.OBV(intel['Intel_Close'],intel['Intel_Volume'])

intel['WilliamsR'] = talib.WILLR(intel['Intel_High'], intel['Intel_Low'], intel['Intel_Close'], timeperiod=14)


intel["MOMENT_10"] = talib.MOM(intel['Intel_Close'], timeperiod=10)
intel["MOMENT_20"] = talib.MOM(intel['Intel_Close'], timeperiod=20)

intel["ROC_10"] = talib.ROC(intel['Intel_Close'], timeperiod=10)
intel["ROC_20"] = talib.ROC(intel['Intel_Close'], timeperiod=20)

intel['Trix'] = talib.TRIX(intel['Intel_Close'], timeperiod=30)



In [13]:
df = pd.merge(intel, peers, on='Date', how='inner')


In [14]:
#To set the target I calculated the close-open difference
df["Target_BASE"]=df.Intel_Close - df.Intel_Open

In [15]:
# Then target setted binary according to the difference is positive or negative
df['Target'] = df['Target_BASE'].apply(lambda x: 0 if x < 0 else 1)

In [16]:
# SHIFTING 1 TO PREVENT LEAKAGE!!
df['Target'] = df['Target'].shift(1)
df['Target_BASE'] =df['Target_BASE'].shift(1)


In [17]:
df=df.bfill()

In [18]:
data_main=df.copy()

In [19]:
data_main

,Intel_Open,Intel_High,Intel_Low,Intel_Close,Intel_Adj Close,Intel_Volume,Intel_Return,EMA_10,EMA_12,EMA_50,EMA_100,SMA_10,SMA_12,SMA_50,SMA_100,STD_10,STD_12,STD_50,STD_100,ADX_14,ADX_21,RSI_14,RSI_21,MACD,MACDSignal,MACDHist,UPPER_BBand_20,MID_BBand_20,LOWER_BBand_20,Volatility,STOCH_K,STOCH_D,CCI_14,CCI_20,OBV,WilliamsR,MOMENT_10,MOMENT_20,ROC_10,ROC_20,Trix,000001.SS,399001.SZ,AAPL,ADBE,AMD,AVGO,CSCO,IMOEX.ME,MSFT,...,Return_000001.SS,Return_399001.SZ,Return_AAPL,Return_ADBE,Return_AMD,Return_AVGO,Return_CSCO,Return_IMOEX.ME,Return_MSFT,Return_MU,Return_NVDA,Return_NXPI,Return_ORCL,Return_QCOM,Return_SSNLF,Return_TSM,Return_TXN,Return_^AORD,Return_^AXJO,Return_^BFX,Return_^BSESN,Return_^BUK100P,Return_^BVSP,Return_^DJI,Return_^FCHI,Return_^FTSE,Return_^GDAXI,Return_^GSPC,Return_^GSPTSE,Return_^HSI,Return_^IXIC,Return_^JKSE,Return_^JN0U.JO,Return_^KLSE,Return_^KS11,Return_^MERV,Return_^MXX,Return_^N100,Return_^N225,Return_^NYA,Return_^NZ50,Return_^RUT,Return_^STI,Return_^STOXX50E,Return_^TA125.TA,Return_^TWII,Return_^VIX,Return_^XAX,Target_BASE,Target
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-01-03,51.650002,53.230000,51.599998,53.209999,49.350483,33857600,-0.001316,54.516999,54.462499,49.590600,47.924300,54.516999,54.462499,49.5906,47.9243,1.022311,0.959088,2.930448,3.076405,16.472383,18.220351,43.294616,39.014250,-1.759980,-1.915761,0.155781,57.646486,52.842999,48.039513,1.289997,77.108971,75.841475,-127.145951,-153.836562,33857600.0,-90.598239,1.549999,-4.259998,2.912985,-8.006011,-0.124921,3632.330078,14791.309570,179.273621,564.369995,150.240005,62.370541,57.983242,3852.500000,326.940796,...,-0.010228,-0.017953,-0.012692,-0.018374,-0.038738,0.011457,-0.024526,0.005448,-0.017147,0.006162,-0.027589,0.007136,0.010694,0.005478,0.0,0.035714,0.001049,-0.003431,-0.003162,0.006212,0.011367,0.015950,-0.003926,0.005866,0.013882,0.001559,0.008232,-0.000630,-0.009267,0.000648,-0.013269,0.004511,0.015858,-0.004616,-0.011799,-0.003424,0.001431,0.008330,0.001036,0.006424,-0.012727,-0.001624,0.014957,0.008264,0.006637,0.014003,0.018675,0.009917,1.559998,1.0
2022-01-04,53.570000,53.939999,52.650002,53.139999,49.285557,45681400,-0.001316,54.516999,54.462499,49.590600,47.924300,54.516999,54.462499,49.5906,47.9243,1.022311,0.959088,2.930448,3.076405,16.472383,18.220351,43.294616,39.014250,-1.759980,-1.915761,0.155781,57.646486,52.842999,48.039513,1.289997,77.108971,75.841475,-127.145951,-153.836562,-11823800.0,-90.598239,1.549999,-4.259998,2.912985,-8.006011,-0.124921,3632.330078,14791.309570,176.998352,554.000000,144.419998,63.085148,56.561138,3873.489990,321.334717,...,-0.010228,-0.017953,-0.012692,-0.018374,-0.038738,0.011457,-0.024526,0.005448,-0.017147,0.006162,-0.027589,0.007136,0.010694,0.005478,0.0,0.035714,0.001049,-0.003431,-0.003162,0.006212,0.011367,0.015950,-0.003926,0.005866,0.013882,0.001559,0.008232,-0.000630,-0.009267,0.000648,-0.013269,0.004511,0.015858,-0.004616,-0.011799,-0.003424,0.001431,0.008330,0.001036,0.006424,-0.012727,-0.001624,0.014957,0.008264,0.000805,0.014003,0.018675,0.009917,1.559998,1.0
2022-01-05,54.189999,56.169998,53.830002,53.869999,49.962608,59109300,0.013737,54.516999,54.462499,49.590600,47.924300,54.516999,54.462499,49.5906,47.9243,1.022311,0.959088,2.930448,3.076405,16.472383,18.220351,43.294616,39.014250,-1.759980,-1.915761,0.155781,57.646486,52.842999,48.039513,3.029999,77.108971,75.841475,-127.145951,-153.836562,47285500.0,-90.598239,1.549999,-4.259998,2.912985,-8.006011,-0.124921,3595.179932,14525.759766,172.290207,514.429993,136.149994,60.459896,55.665394,3815.050049,308.999420,...,-0.010228,-0.017953,-0.026600,-0.071426,-0.057264,-0.041614,-0.015837,-0.015087,-0.038388,-0.020137,-0.057562,-0.035642,-0.026790,-0.003899,0.0,-0.047526,-0.020860,-0.003431,-0.003162,-0.004032,0.006135,0.002944,-0.024229,-0.010667,0.008057,0.001559,0.007376,-0.019393,-0.009267,-0.016427,-0.033448,-0.004940,0.015189,0.003924,-0.011799,-0.017002,0

#### After data preparation and target set I can go with the modeling 

# ROLLING WINDOW CROSS-VALIDATION

In [20]:
data = data_main.drop(columns=['Target_BASE'])

# Defining feature and target
X = data.drop(columns=['Target'])
Y = data['Target']

# Normalization of data
scaler = MinMaxScaler(feature_range=(0, 1))

# Feature selection functions

def select_features_corr(X_train, Y_train, varnum):
    corr_matrix = data.corr()

    corr_with_target = corr_matrix.iloc[:-1, -1]

    sorted_corr = corr_with_target.abs().sort_values(ascending=False)

    selected_features_name = sorted_corr.head(varnum).index

    selected_features = [X.columns.get_loc(feature) for feature in selected_features_name]
    print(f"Selected features (CorrelationFS): {selected_features}")

    return selected_features


def select_features_chi_square(X_train, Y_train, varnum):
    selector = SelectKBest(chi2, k=varnum).fit(X_train, Y_train)

    selected_features = selector.get_support(indices=True)

    print(f"Selected features (ChiSquareFS): {selected_features}")
    return selected_features

def select_features_rf(X_train, Y_train, varnumn_estimators=50):
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, Y_train)
    model_selector = SelectFromModel(rf, max_features=varnum, prefit=True)
    selected_features = model_selector.get_support(indices=True)
    feature_importances = rf.feature_importances_
    sorted_indices = np.argsort(feature_importances)[::-1]

    # Select the most important 'varnum' number feature
    selected_features = sorted_indices[:varnum]
    print(f"Selected features (RF): {selected_features}")
    print(f"Feature importances: {feature_importances[selected_features]}")

    return selected_features

def select_features_xgb(X_train, Y_train, varnum):
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42,n_estimator=100,max_depth=6,learning_rate=0.3)
    xgb.fit(X_train, Y_train)
    model_selector = SelectFromModel(xgb, max_features=varnum, prefit=True)
    selected_features = model_selector.get_support(indices=True)
    feature_importances = xgb.feature_importances_
    
    # Select the most important 'varnum' number feature
    
    sorted_indices = np.argsort(feature_importances)[::-1]
    selected_features = sorted_indices[:varnum]
    print(f"Selected features (XGB): {selected_features}")
    print(f"Feature importances: {feature_importances[selected_features]}")

    return selected_features

# Model functions
def create_cart_model():
    return DecisionTreeClassifier(random_state=42)

def create_rf_model():
    return RandomForestClassifier(random_state=42)

def create_xgb_model():
    return XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

def create_lda_model():
    return LinearDiscriminantAnalysis()


# Custom time series splitting function
def custom_time_series_split(data, train_period_months=6, test_period_months=1):
    periods = []
    start_date = data.index.min()
    end_date = data.index.max()

    current_date = start_date

    while current_date + relativedelta(months=train_period_months + test_period_months) <= end_date:
        train_start = current_date
        train_end = train_start + relativedelta(months=train_period_months) - pd.DateOffset(days=1)
        test_start = train_end + pd.DateOffset(days=1)
        test_end = test_start + relativedelta(months=test_period_months) - pd.DateOffset(days=1)

        train_indices = data.loc[train_start:train_end].index
        test_indices = data.loc[test_start:test_end].index

        periods.append((train_indices, test_indices))

        current_date = current_date + relativedelta(months=1)

    return periods

def expanding_time_series_split(data, initial_train_period_months=6, test_period_months=1):
    periods = []
    start_date = data.index.min()
    end_date = data.index.max()

    current_date = start_date + relativedelta(months=initial_train_period_months)

    while current_date + relativedelta(months=test_period_months) <= end_date:
        train_start = start_date
        train_end = current_date - pd.DateOffset(days=1)
        test_start = train_end + pd.DateOffset(days=1)
        test_end = test_start + relativedelta(months=test_period_months) - pd.DateOffset(days=1)

        train_indices = data.loc[train_start:train_end].index
        test_indices = data.loc[test_start:test_end].index

        periods.append((train_indices, test_indices))

        current_date = current_date + relativedelta(months=1)

    return periods

# Return calculation function
def calculate_returns(data_main, predictions, start_date, end_date):
    period_data = data_main.loc[start_date:end_date]
    period_returns = period_data['Target_BASE']

    correct_predictions = (predictions == period_data['Target'])
    incorrect_predictions = ~correct_predictions

    returns = np.where(correct_predictions, abs(period_returns), -abs(period_returns))
    return returns.sum()

def calculate_metrics(data_main, predictions, start_date, end_date):
    period_data = data_main.loc[start_date:end_date]
    period_returns = period_data['Target_BASE']

    correct_predictions = (predictions == period_data['Target'])
    incorrect_predictions = ~correct_predictions

    returns = np.where(correct_predictions, abs(period_returns), -abs(period_returns))

    return_df = pd.DataFrame(returns, index=period_data.index)

    return_monthly= return_df.resample('M').sum()
    min_return_monthly =return_monthly.min().iloc[0]
    neg_months = (return_monthly < 0).sum().iloc[0].sum()


    return_weekly = return_df.resample('W').sum()
    min_return_weekly = return_weekly.min().iloc[0]
    neg_weeks = (return_weekly < 0).sum().iloc[0].sum()


    long_preds=np.where(predictions > 0.5,1,0)
    short_preds=np.where(predictions < 0.5,1,0)

    long_f1=f1_score(period_data['Target'],long_preds)
    short_f1=f1_score(period_data['Target'],short_preds)

    return return_monthly,min_return_monthly,neg_months,return_weekly,min_return_weekly,neg_weeks, long_f1, short_f1


# Trend calculation function
def calculate_trend(values):
    if len(values) < 2:
        return np.nan
    x = np.arange(len(values))
    slope, _, _, _, _ = linregress(x, values)
    return slope

# Evaluation
varnums = [5,10,15]
selection_methods = {
    'CORR': select_features_corr,
     'CHI2': select_features_chi_square,
     'RF': select_features_rf,
     'XGB': select_features_xgb,
}
results_df_roll = pd.DataFrame(columns=['Fold', 'Model', 'Feature Selection Method', 'VarNum', 'Train Period', 'Test Period', 'Train Accuracy', 'Test Accuracy', 'Train Return Mean', 'Test Return Mean', 'Selected Features'])

#periods = expanding_time_series_split(data, initial_train_period_months=6, test_period_months=1)
periods = custom_time_series_split(data, train_period_months=6, test_period_months=1)

fold = 0

selected_features_count = {}

for varnum in varnums:
    for method_name, selection_method in selection_methods.items():
        for model_name, create_model in [('CART', create_cart_model), ('RF', create_rf_model), ('XGB', create_xgb_model),('LDA', create_lda_model)]:
            fold = 0
            for train_indices, test_indices in periods:
                train_indices = data.index.get_indexer(train_indices)
                test_indices = data.index.get_indexer(test_indices)
                                
                X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
                Y_train, Y_test = Y.iloc[train_indices], Y.iloc[test_indices]
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
                
                selected_features = selection_method(X_train, Y_train, varnum)
                print(f"Fold {fold}, VarNum {varnum}, Method {method_name}, Model {model_name}, Selected Features: {selected_features}")

                X_train_selected = X_train[:, selected_features]
                X_test_selected = X_test[:, selected_features]

                selected_feature_names = X.columns[selected_features].tolist()

                for feature in selected_feature_names:
                    if feature not in selected_features_count:
                        selected_features_count[feature] = 0
                    selected_features_count[feature] += 1

                model = create_model()
                model.fit(X_train_selected, Y_train)

                train_predictions = model.predict(X_train_selected)
                test_predictions = model.predict(X_test_selected)

                train_acc = np.mean(train_predictions == Y_train)
                test_acc = np.mean(test_predictions == Y_test)

                print(f"Fold {fold}, VarNum {varnum}, Method {method_name}, Model {model_name}, Train Acc: {train_acc}, Test Acc: {test_acc}")

                train_start_date = data.index[train_indices].min()
                train_end_date = data.index[train_indices].max()
                test_start_date = data.index[test_indices].min()
                test_end_date = data.index[test_indices].max()

                train_return_mean = calculate_returns(data_main, train_predictions, train_start_date, train_end_date)
                test_return_mean = calculate_returns(data_main, test_predictions, test_start_date, test_end_date)
                return_monthly,min_return_monthly,neg_months,return_weekly,min_return_weekly,neg_weeks,long_f1,short_f1 = calculate_metrics(data_main, test_predictions, test_start_date, test_end_date)


                fold += 1
                results_df_roll = pd.concat([results_df_roll, pd.DataFrame({
                    'Fold': [fold],
                    'Model': [model_name],
                    'Feature Selection Method': [method_name],
                    'VarNum': [varnum],
                    'Train Period': [f'{train_start_date.date()} - {train_end_date.date()}'],
                    'Test Period': [f'{test_start_date.date()} - {test_end_date.date()}'],
                    'Train Accuracy': [train_acc],
                    'Test Accuracy': [test_acc],
                    'Train Return Mean': [train_return_mean],
                    'Test Return Mean': [test_return_mean],
                    'Selected Features': [selected_feature_names],
                    'Monthly Min Return':[min_return_monthly],
                    'Negative Month Number':[neg_months],
                    'Weekly Minimum Return':[min_return_weekly],
                    'Negative Week Number':[neg_weeks],
                    'Long F1':[long_f1],
                    'Short F1':[short_f1]

                })], ignore_index=True)

# Calculate the number of negative return periods
results_df_roll['Negative Train Return'] = results_df_roll['Train Return Mean'] < 0
results_df_roll['Negative Test Return'] = results_df_roll['Test Return Mean'] < 0

# Trend calculation
results_df_roll['Test Accuracy Trend'] = results_df_roll.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Accuracy'].transform(lambda x: calculate_trend(x[-25:]))
results_df_roll['Test Return Mean Trend'] = results_df_roll.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Return Mean'].transform(lambda x: calculate_trend(x[-25:]))

# Calculate the mean and std of test accuracies
mean_test_acc = results_df_roll.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Accuracy'].mean()
std_test_acc = results_df_roll.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Accuracy'].std()

# Create a pivot table for the results
pivot_table_roll = results_df_roll.pivot_table(
    index=['Model', 'Feature Selection Method', 'VarNum'],

    values=['Test Accuracy', 'Test Return Mean', 'Negative Test Return', 'Test Accuracy Trend', 'Test Return Mean Trend','Monthly Min Return','Negative Month Number',
            'Weekly Minimum Return','Negative Week Number','Long F1','Short F1'],
    aggfunc={
        'Test Accuracy': [np.mean, np.std, np.min],
        'Test Return Mean': [np.mean, np.std],
        'Negative Test Return': np.sum,
        'Test Accuracy Trend': np.mean,
        'Test Return Mean Trend': np.mean,
        'Monthly Min Return':np.mean,
        'Negative Month Number':np.mean,
        'Weekly Minimum Return':np.mean,
        'Negative Week Number':np.mean,
        'Long F1':np.mean,
        'Short F1':np.mean
    }
).reset_index()



# Flatten the multi-level columns
pivot_table_roll.columns = [' '.join(col).strip() for col in pivot_table_roll.columns.values]
pivot_table_roll['Periods'] = len(periods)

# Create placeholder for Top Features
pivot_table_roll[['Top Feature 1', 'Top Feature 2', 'Top Feature 3', 'Top Feature 4', 'Top Feature 5']] = ''

# Find the 5 most frequently used variables in the result_df table and add them to the pivot_table
for idx, row in pivot_table_roll.iterrows():
    model_name = row['Model']
    method_name = row['Feature Selection Method']
    varnum = row['VarNum']

    filtered_df = results_df_roll[(results_df_roll['Model'] == model_name) &
                             (results_df_roll['Feature Selection Method'] == method_name) &
                             (results_df_roll['VarNum'] == varnum)]

    feature_counts = {}
    for features in filtered_df['Selected Features']:
        for feature in features:
            if feature not in feature_counts:
                feature_counts[feature] = 0
            feature_counts[feature] += 1

    top_features = pd.Series(feature_counts).nlargest(5).index.tolist()

    for i, feature in enumerate(top_features):
        pivot_table_roll.at[idx, f'Top Feature {i+1}'] = feature



Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 0, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 0, VarNum 5, Method CORR, Model CART, Train Acc: 0.968, Test Acc: 0.6666666666666666
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 1, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 1, VarNum 5, Method CORR, Model CART, Train Acc: 1.0, Test Acc: 0.6521739130434783
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 2, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 2, VarNum 5, Method CORR, Model CART, Train Acc: 1.0, Test Acc: 0.5789473684210527
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 3, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 3, VarNum 5, Method CORR, Model CART, Train Acc: 1.0, Test Acc: 0.6086956521739131
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 4, VarNum 5, Method CORR, Mod

Fold 14, VarNum 5, Method CORR, Model RF, Train Acc: 1.0, Test Acc: 0.6
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 15, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 15, VarNum 5, Method CORR, Model RF, Train Acc: 1.0, Test Acc: 0.6956521739130435
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 16, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 16, VarNum 5, Method CORR, Model RF, Train Acc: 1.0, Test Acc: 0.65
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 17, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 17, VarNum 5, Method CORR, Model RF, Train Acc: 1.0, Test Acc: 0.55
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 18, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 18, VarNum 5, Method CORR, Model RF, Train Acc: 1.0, Test Acc: 0.6818181818181818
Selected features (CorrelationFS): [30, 32, 33, 35, 21]


Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 7, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 7, VarNum 5, Method CORR, Model LDA, Train Acc: 0.6850393700787402, Test Acc: 0.7894736842105263
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 8, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 8, VarNum 5, Method CORR, Model LDA, Train Acc: 0.7479674796747967, Test Acc: 0.7619047619047619
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 9, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 9, VarNum 5, Method CORR, Model LDA, Train Acc: 0.744, Test Acc: 0.6666666666666666
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 10, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 10, VarNum 5, Method CORR, Model LDA, Train Acc: 0.7398373983739838, Test Acc: 0.7272727272727273
Selected features (CorrelationFS): [30, 32, 33, 35,

Fold 0, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.7619047619047619
Selected features (ChiSquareFS): [ 30  35 106 107 127]
Fold 1, VarNum 5, Method CHI2, Model RF, Selected Features: [ 30  35 106 107 127]
Fold 1, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.5217391304347826
Selected features (ChiSquareFS): [30 31 32 33 35]
Fold 2, VarNum 5, Method CHI2, Model RF, Selected Features: [30 31 32 33 35]
Fold 2, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.631578947368421
Selected features (ChiSquareFS): [ 30  35 106 107 127]
Fold 3, VarNum 5, Method CHI2, Model RF, Selected Features: [ 30  35 106 107 127]
Fold 3, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.782608695652174
Selected features (ChiSquareFS): [ 30  32  35 107 127]
Fold 4, VarNum 5, Method CHI2, Model RF, Selected Features: [ 30  32  35 107 127]
Fold 4, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.7619047619047619
Selected features (ChiSquareFS): [

Fold 16, VarNum 5, Method CHI2, Model XGB, Train Acc: 1.0, Test Acc: 0.7
Selected features (ChiSquareFS): [ 21  30  32  35 106]
Fold 17, VarNum 5, Method CHI2, Model XGB, Selected Features: [ 21  30  32  35 106]
Fold 17, VarNum 5, Method CHI2, Model XGB, Train Acc: 1.0, Test Acc: 0.5
Selected features (ChiSquareFS): [ 30  32  35 122 127]
Fold 18, VarNum 5, Method CHI2, Model XGB, Selected Features: [ 30  32  35 122 127]
Fold 18, VarNum 5, Method CHI2, Model XGB, Train Acc: 1.0, Test Acc: 0.5909090909090909
Selected features (ChiSquareFS): [30 32 33 35 38]
Fold 19, VarNum 5, Method CHI2, Model XGB, Selected Features: [30 32 33 35 38]
Fold 19, VarNum 5, Method CHI2, Model XGB, Train Acc: 1.0, Test Acc: 0.6842105263157895
Selected features (ChiSquareFS): [21 30 31 32 35]
Fold 20, VarNum 5, Method CHI2, Model XGB, Selected Features: [21 30 31 32 35]
Fold 20, VarNum 5, Method CHI2, Model XGB, Train Acc: 1.0, Test Acc: 0.7142857142857143
Selected features (ChiSquareFS): [30 31 32 35 55]
Fold

...

Fold 9, VarNum 15, Method XGB, Model RF, Train Acc: 1.0, Test Acc: 0.47619047619047616
[15:23:40] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [ 35  50 132  33  69  56  42 111  65  27  91  98   6 128  44]
Feature importances: [0.08362651 0.04993495 0.04730373 0.04295497 0.04127013 0.04062516
 0.04024033 0.03461215 0.03025046 0.02787456 0.02475373 0.02299904
 0.02182731 0.0201506  0.02010849]
Fold 10, VarNum 15, Method XGB, Model RF, Selected Features: [ 35  50 132  33  69  56  42 111  65  27  91  98   6 128  44]
Fold 10, VarNum 15, Method XGB, Model RF, Train Acc: 1.0, Test Acc: 0.6363636363636364
[15:23:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB):

Fold 21, VarNum 15, Method XGB, Model RF, Train Acc: 1.0, Test Acc: 0.6818181818181818
[15:23:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [ 54  92  67 105  46  76 134 111  66  57 106 133  78 100 109]
Feature importances: [0.05529401 0.04942611 0.04690247 0.03984741 0.03686202 0.03489007
 0.03425908 0.03199412 0.03003381 0.02984594 0.02863571 0.02685573
 0.02517813 0.02097734 0.01903709]
Fold 0, VarNum 15, Method XGB, Model XGB, Selected Features: [ 54  92  67 105  46  76 134 111  66  57 106 133  78 100 109]
Fold 0, VarNum 15, Method XGB, Model XGB, Train Acc: 1.0, Test Acc: 0.5238095238095238
[15:23:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB):

Fold 12, VarNum 15, Method XGB, Model XGB, Train Acc: 1.0, Test Acc: 0.5909090909090909
[15:23:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [125  33  47  26  65 124  32 100  95 128  25  29  20 122 106]
Feature importances: [0.09067244 0.07497578 0.0657309  0.04896316 0.03977192 0.0375411
 0.0296344  0.028289   0.02809397 0.02647186 0.02582458 0.02543353
 0.02503267 0.02181566 0.02006459]
Fold 13, VarNum 15, Method XGB, Model XGB, Selected Features: [125  33  47  26  65 124  32 100  95 128  25  29  20 122 106]
Fold 13, VarNum 15, Method XGB, Model XGB, Train Acc: 1.0, Test Acc: 0.45454545454545453
[15:23:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XG

Selected features (XGB): [ 32  61 106  29  81  92 105  30 104  95  96 107 135  83 129]
Feature importances: [0.06081986 0.06005809 0.05691498 0.05008206 0.04921301 0.03296668
 0.03100923 0.02906076 0.02756156 0.02643843 0.02640826 0.02447952
 0.0238907  0.02016696 0.01899442]
Fold 4, VarNum 15, Method XGB, Model LDA, Selected Features: [ 32  61 106  29  81  92 105  30 104  95  96 107 135  83 129]
Fold 4, VarNum 15, Method XGB, Model LDA, Train Acc: 0.7421875, Test Acc: 0.6190476190476191
[15:23:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [ 37  63  23  72   6 107  68 106 131   0  32  25  98 108 101]
Feature importances: [0.12117104 0.08826667 0.0610269  0.05738462 0.03618091 0.03251973
 0.02970542 0.02945096 0.02717068 0.02702343 0.02423287 0.0239141
 0.02381011 0.01915811 0.01777912]
Fold 5, VarNum 15, Method XGB,

Selected features (XGB): [115  88  32 109   7  35   6 107 120   8 118  52  15  29 131]
Feature importances: [0.08838061 0.04788351 0.03626766 0.03557093 0.03512619 0.03208331
 0.03144437 0.02985148 0.0276808  0.02635875 0.02586114 0.02488816
 0.02475854 0.02421924 0.02399355]
Fold 16, VarNum 15, Method XGB, Model LDA, Selected Features: [115  88  32 109   7  35   6 107 120   8 118  52  15  29 131]
Fold 16, VarNum 15, Method XGB, Model LDA, Train Acc: 0.7421875, Test Acc: 0.5
[15:23:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [112   2  36 113  74  15  11  21  90  65 105  47  61 102  66]
Feature importances: [0.08225743 0.06962254 0.04561475 0.04289938 0.03476458 0.02770768
 0.02723359 0.02665008 0.02389776 0.02333322 0.02038334 0.01794489
 0.01787452 0.01743047 0.01716154]
Fold 17, VarNum 15, Method XGB, Model LDA,

In [21]:
pivot_table_roll

,Model,Feature Selection Method,VarNum,Long F1 mean,Monthly Min Return mean,Negative Month Number mean,Negative Test Return sum,Negative Week Number mean,Short F1 mean,Test Accuracy amin,Test Accuracy mean,Test Accuracy std,Test Accuracy Trend mean,Test Return Mean mean,Test Return Mean std,Test Return Mean Trend mean,Weekly Minimum Return mean,Periods,Top Feature 1,Top Feature 2,Top Feature 3,Top Feature 4,Top Feature 5
0,CART,CHI2,5,0.582865,0.219092,0.727273,4,1.727273,0.404762,0.363636,0.575547,0.099888,-0.006348,2.527727,2.510014,-0.091807,-1.110000,22,STOCH_K,WilliamsR,CCI_14,CCI_20,Return_^N225
1,CART,CHI2,10,0.615319,1.165910,0.227273,1,1.045455,0.369625,0.368421,0.623306,0.083067,-0.003039,3.916818,2.954734,-0.080672,-0.668183,22,STOCH_K,CCI_20,WilliamsR,CCI_14,RSI_14
2,CART,CHI2,15,0.572240,0.858182,0.545455,4,1.500000,0.407248,0.318182,0.591092,0.123076,-0.014090,2.768639,3.219808,-0.384647,-0.860454,22,STOCH_K,CCI_20,WilliamsR,CCI_14,RSI_14
3,CART,CORR,5,0.546121,-0.067272,0.590909,6,1.681818,0.442854,0.318182,0.570561,0.094060,-0.004886,2.151366,2.588190,-0.099215,-1.242727,22,STOCH_K,CCI_14,CCI_20,WilliamsR,RSI_14
4,CART,CORR,10,0.592467,0.692727,0.545455,4,1.409091,0.388001,0.409091,0.605465,0.122911,-0.003695,3.026820,2.835909,-0.019497,-0.852727,22,STOCH_K,CCI_14,CCI_20,WilliamsR,RSI_14
5,CART,CORR,15,0.604854,0.671365,0.500000,3,1.454545,0.394080,0.454545,0.596827,0.091604,-0.007736,2.556820,2.715199,-0.284285,-0.912727,22,STOCH_K,CCI_14,CCI_20,WilliamsR,RSI_14
6,CART,RF,5,0.575937,0.383637,0.590909,4,1.681818,0.414290,0.350000,0.562428,0.100915,-0.003966,2.369546,2.616207,-0.142874,-0.933636,22,STOCH_K,CCI_20,Return_^AXJO,CCI_14,Return_^AORD
7,CART,RF,10,0.567338,0.037271,0.727273,7,1.772727,0.409857,0.285714,0.572598,0.140063,-0.010372,1.998635,3.897937,-0.253343,-1.106819,22,STOCH_K,CCI_20,CCI_14,Return_^TWII,Return_^AORD
8,CART,RF,15,0.620156,0.627727,0.545455,5,1.636364,0.362147,0.333333,0.590089,0.123785,-0.010156,2.599545,3.873750,-0.303992,-0.891818,22,STOCH_K,CCI_20,CCI_14,Return_^AORD,Return_^TWII
9,CART,XGB,5,0.480400,-0.931364,0.818182,7,2.181818,0.460053,0.300000,0.521328,0.108768,-0.002030,0.474092,3.376659,-0.046940,-1.621363,22,CCI_20,WilliamsR,Return_^DJI,Return_ADBE,Return_TXN


# EXPANDIG WINDOW CROSS VALIDATION

In [22]:
data = data_main.drop(columns=['Target_BASE'])

# Defining feature and target
X = data.drop(columns=['Target'])
Y = data['Target']

# Normalization of data
scaler = MinMaxScaler(feature_range=(0, 1))

# Feature selection functions

def select_features_corr(X_train, Y_train, varnum):
    corr_matrix = data.corr()

    corr_with_target = corr_matrix.iloc[:-1, -1]

    sorted_corr = corr_with_target.abs().sort_values(ascending=False)

    selected_features_name = sorted_corr.head(varnum).index

    selected_features = [X.columns.get_loc(feature) for feature in selected_features_name]
    print(f"Selected features (CorrelationFS): {selected_features}")

    return selected_features


def select_features_chi_square(X_train, Y_train, varnum):
    selector = SelectKBest(chi2, k=varnum).fit(X_train, Y_train)

    selected_features = selector.get_support(indices=True)

    print(f"Selected features (ChiSquareFS): {selected_features}")
    return selected_features

def select_features_rf(X_train, Y_train, varnumn_estimators=50):
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, Y_train)
    model_selector = SelectFromModel(rf, max_features=varnum, prefit=True)
    selected_features = model_selector.get_support(indices=True)
    feature_importances = rf.feature_importances_
    sorted_indices = np.argsort(feature_importances)[::-1]

    # Select the most important 'varnum' number feature
    selected_features = sorted_indices[:varnum]
    print(f"Selected features (RF): {selected_features}")
    print(f"Feature importances: {feature_importances[selected_features]}")

    return selected_features

def select_features_xgb(X_train, Y_train, varnum):
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42,n_estimator=100,max_depth=6,learning_rate=0.3)
    xgb.fit(X_train, Y_train)
    model_selector = SelectFromModel(xgb, max_features=varnum, prefit=True)
    selected_features = model_selector.get_support(indices=True)
    feature_importances = xgb.feature_importances_
    
    # Select the most important 'varnum' number feature
    
    sorted_indices = np.argsort(feature_importances)[::-1]
    selected_features = sorted_indices[:varnum]
    print(f"Selected features (XGB): {selected_features}")
    print(f"Feature importances: {feature_importances[selected_features]}")

    return selected_features

# Model functions
def create_cart_model():
    return DecisionTreeClassifier(random_state=42)

def create_rf_model():
    return RandomForestClassifier(random_state=42)

def create_xgb_model():
    return XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

def create_lda_model():
    return LinearDiscriminantAnalysis()


# Custom time series splitting function
def custom_time_series_split(data, train_period_months=6, test_period_months=1):
    periods = []
    start_date = data.index.min()
    end_date = data.index.max()

    current_date = start_date

    while current_date + relativedelta(months=train_period_months + test_period_months) <= end_date:
        train_start = current_date
        train_end = train_start + relativedelta(months=train_period_months) - pd.DateOffset(days=1)
        test_start = train_end + pd.DateOffset(days=1)
        test_end = test_start + relativedelta(months=test_period_months) - pd.DateOffset(days=1)

        train_indices = data.loc[train_start:train_end].index
        test_indices = data.loc[test_start:test_end].index

        periods.append((train_indices, test_indices))

        current_date = current_date + relativedelta(months=1)

    return periods

def expanding_time_series_split(data, initial_train_period_months=6, test_period_months=1):
    periods = []
    start_date = data.index.min()
    end_date = data.index.max()

    current_date = start_date + relativedelta(months=initial_train_period_months)

    while current_date + relativedelta(months=test_period_months) <= end_date:
        train_start = start_date
        train_end = current_date - pd.DateOffset(days=1)
        test_start = train_end + pd.DateOffset(days=1)
        test_end = test_start + relativedelta(months=test_period_months) - pd.DateOffset(days=1)

        train_indices = data.loc[train_start:train_end].index
        test_indices = data.loc[test_start:test_end].index

        periods.append((train_indices, test_indices))

        current_date = current_date + relativedelta(months=1)

    return periods

# Return calculation function
def calculate_returns(data_main, predictions, start_date, end_date):
    period_data = data_main.loc[start_date:end_date]
    period_returns = period_data['Target_BASE']

    correct_predictions = (predictions == period_data['Target'])
    incorrect_predictions = ~correct_predictions

    returns = np.where(correct_predictions, abs(period_returns), -abs(period_returns))
    return returns.sum()

def calculate_metrics(data_main, predictions, start_date, end_date):
    period_data = data_main.loc[start_date:end_date]
    period_returns = period_data['Target_BASE']

    correct_predictions = (predictions == period_data['Target'])
    incorrect_predictions = ~correct_predictions

    returns = np.where(correct_predictions, abs(period_returns), -abs(period_returns))

    return_df = pd.DataFrame(returns, index=period_data.index)

    return_monthly= return_df.resample('M').sum()
    min_return_monthly =return_monthly.min().iloc[0]
    neg_months = (return_monthly < 0).sum().iloc[0].sum()


    return_weekly = return_df.resample('W').sum()
    min_return_weekly = return_weekly.min().iloc[0]
    neg_weeks = (return_weekly < 0).sum().iloc[0].sum()

    long_preds=np.where(predictions > 0.5,1,0)
    short_preds=np.where(predictions < 0.5,1,0)

    long_f1=f1_score(period_data['Target'],long_preds)
    short_f1=f1_score(period_data['Target'],short_preds)

    return return_monthly,min_return_monthly,neg_months,return_weekly,min_return_weekly,neg_weeks,long_f1,short_f1


# Trend calculation function
def calculate_trend(values):
    if len(values) < 2:
        return np.nan
    x = np.arange(len(values))
    slope, _, _, _, _ = linregress(x, values)
    return slope

# Evaluation
varnums = [5,10,15]
selection_methods = {
    'CORR': select_features_corr,
     'CHI2': select_features_chi_square,
     'RF': select_features_rf,
     'XGB': select_features_xgb,
}
results_df_exp = pd.DataFrame(columns=['Fold', 'Model', 'Feature Selection Method', 'VarNum', 'Train Period', 'Test Period', 'Train Accuracy', 'Test Accuracy', 'Train Return Mean', 'Test Return Mean', 'Selected Features'])

periods = expanding_time_series_split(data, initial_train_period_months=6, test_period_months=1)
#periods = custom_time_series_split(data, train_period_months=6, test_period_months=1)

fold = 0

selected_features_count = {}

for varnum in varnums:
    for method_name, selection_method in selection_methods.items():
        for model_name, create_model in [('CART', create_cart_model), ('RF', create_rf_model), ('XGB', create_xgb_model),('LDA', create_lda_model)]:
            fold = 0
            for train_indices, test_indices in periods:
                train_indices = data.index.get_indexer(train_indices)
                test_indices = data.index.get_indexer(test_indices)
                                
                X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
                Y_train, Y_test = Y.iloc[train_indices], Y.iloc[test_indices]
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
                
                selected_features = selection_method(X_train, Y_train, varnum)
                print(f"Fold {fold}, VarNum {varnum}, Method {method_name}, Model {model_name}, Selected Features: {selected_features}")

                X_train_selected = X_train[:, selected_features]
                X_test_selected = X_test[:, selected_features]

                selected_feature_names = X.columns[selected_features].tolist()

                for feature in selected_feature_names:
                    if feature not in selected_features_count:
                        selected_features_count[feature] = 0
                    selected_features_count[feature] += 1

                model = create_model()
                model.fit(X_train_selected, Y_train)

                train_predictions = model.predict(X_train_selected)
                test_predictions = model.predict(X_test_selected)

                train_acc = np.mean(train_predictions == Y_train)
                test_acc = np.mean(test_predictions == Y_test)

                print(f"Fold {fold}, VarNum {varnum}, Method {method_name}, Model {model_name}, Train Acc: {train_acc}, Test Acc: {test_acc}")

                train_start_date = data.index[train_indices].min()
                train_end_date = data.index[train_indices].max()
                test_start_date = data.index[test_indices].min()
                test_end_date = data.index[test_indices].max()

                train_return_mean = calculate_returns(data_main, train_predictions, train_start_date, train_end_date)
                test_return_mean = calculate_returns(data_main, test_predictions, test_start_date, test_end_date)
                return_monthly,min_return_monthly,neg_months,return_weekly,min_return_weekly,neg_weeks,long_f1,short_f1 = calculate_metrics(data_main, test_predictions, test_start_date, test_end_date)


                fold += 1
                results_df_exp = pd.concat([results_df_exp, pd.DataFrame({
                    'Fold': [fold],
                    'Model': [model_name],
                    'Feature Selection Method': [method_name],
                    'VarNum': [varnum],
                    'Train Period': [f'{train_start_date.date()} - {train_end_date.date()}'],
                    'Test Period': [f'{test_start_date.date()} - {test_end_date.date()}'],
                    'Train Accuracy': [train_acc],
                    'Test Accuracy': [test_acc],
                    'Train Return Mean': [train_return_mean],
                    'Test Return Mean': [test_return_mean],
                    'Selected Features': [selected_feature_names],
                    'Monthly Min Return':[min_return_monthly],
                    'Negative Month Number':[neg_months],
                    'Weekly Minimum Return':[min_return_weekly],
                    'Negative Week Number':[neg_weeks],
                    'Long F1':[long_f1],
                    'Short F1':[short_f1]

                })], ignore_index=True)

# Calculate the number of negative return periods
results_df_exp['Negative Train Return'] = results_df_exp['Train Return Mean'] < 0
results_df_exp['Negative Test Return'] = results_df_exp['Test Return Mean'] < 0

# Trend calculation
results_df_exp['Test Accuracy Trend'] = results_df_exp.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Accuracy'].transform(lambda x: calculate_trend(x[-25:]))
results_df_exp['Test Return Mean Trend'] = results_df_exp.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Return Mean'].transform(lambda x: calculate_trend(x[-25:]))

# Calculate the mean and std of test accuracies
mean_test_acc = results_df_exp.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Accuracy'].mean()
std_test_acc = results_df_exp.groupby(['Model', 'Feature Selection Method', 'VarNum'])['Test Accuracy'].std()

# Create a pivot table for the results
pivot_table_exp = results_df_exp.pivot_table(
    index=['Model', 'Feature Selection Method', 'VarNum'],

    values=['Test Accuracy', 'Test Return Mean', 'Negative Test Return', 'Test Accuracy Trend', 'Test Return Mean Trend','Monthly Min Return','Negative Month Number',
            'Weekly Minimum Return','Negative Week Number','Long F1','Short F1'],
    aggfunc={
        'Test Accuracy': [np.mean, np.std, np.min],
        'Test Return Mean': [np.mean, np.std],
        'Negative Test Return': np.sum,
        'Test Accuracy Trend': np.mean,
        'Test Return Mean Trend': np.mean,
        'Monthly Min Return':np.mean,
        'Negative Month Number':np.mean,
        'Weekly Minimum Return':np.mean,
        'Negative Week Number':np.mean,
        'Long F1':np.mean,
        'Short F1':np.mean
    }
).reset_index()



# Flatten the multi-level columns
pivot_table_exp.columns = [' '.join(col).strip() for col in pivot_table_exp.columns.values]
pivot_table_exp['Periods'] = len(periods)

# Create placeholder for Top Features
pivot_table_exp[['Top Feature 1', 'Top Feature 2', 'Top Feature 3', 'Top Feature 4', 'Top Feature 5']] = ''

# Find the 5 most frequently used variables in the result_df table and add them to the pivot_table
for idx, row in pivot_table_exp.iterrows():
    model_name = row['Model']
    method_name = row['Feature Selection Method']
    varnum = row['VarNum']

    filtered_df = results_df_exp[(results_df_exp['Model'] == model_name) &
                             (results_df_exp['Feature Selection Method'] == method_name) &
                             (results_df_exp['VarNum'] == varnum)]

    feature_counts = {}
    for features in filtered_df['Selected Features']:
        for feature in features:
            if feature not in feature_counts:
                feature_counts[feature] = 0
            feature_counts[feature] += 1

    top_features = pd.Series(feature_counts).nlargest(5).index.tolist()

    for i, feature in enumerate(top_features):
        pivot_table_exp.at[idx, f'Top Feature {i+1}'] = feature



Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 0, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 0, VarNum 5, Method CORR, Model CART, Train Acc: 0.968, Test Acc: 0.6666666666666666
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 1, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 1, VarNum 5, Method CORR, Model CART, Train Acc: 0.9726027397260274, Test Acc: 0.5217391304347826
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 2, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 2, VarNum 5, Method CORR, Model CART, Train Acc: 0.9763313609467456, Test Acc: 0.5263157894736842
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 3, VarNum 5, Method CORR, Model CART, Selected Features: [30, 32, 33, 35, 21]
Fold 3, VarNum 5, Method CORR, Model CART, Train Acc: 0.9787234042553191, Test Acc: 0.5652173913043478
Selected features (CorrelationFS): [30, 32, 3

Fold 12, VarNum 5, Method CORR, Model RF, Train Acc: 0.9893333333333333, Test Acc: 0.5909090909090909
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 13, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 13, VarNum 5, Method CORR, Model RF, Train Acc: 0.9899244332493703, Test Acc: 0.6363636363636364
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 14, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 14, VarNum 5, Method CORR, Model RF, Train Acc: 0.9904534606205251, Test Acc: 0.6
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 15, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 15, VarNum 5, Method CORR, Model RF, Train Acc: 0.9908883826879271, Test Acc: 0.7391304347826086
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 16, VarNum 5, Method CORR, Model RF, Selected Features: [30, 32, 33, 35, 21]
Fold 16, VarNum 5, Method CORR, Model RF, Train Acc: 0.

Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 7, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 7, VarNum 5, Method CORR, Model LDA, Train Acc: 0.6373626373626373, Test Acc: 0.7894736842105263
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 8, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 8, VarNum 5, Method CORR, Model LDA, Train Acc: 0.6506849315068494, Test Acc: 0.7142857142857143
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 9, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 9, VarNum 5, Method CORR, Model LDA, Train Acc: 0.6549520766773163, Test Acc: 0.6666666666666666
Selected features (CorrelationFS): [30, 32, 33, 35, 21]
Fold 10, VarNum 5, Method CORR, Model LDA, Selected Features: [30, 32, 33, 35, 21]
Fold 10, VarNum 5, Method CORR, Model LDA, Train Acc: 0.6526946107784432, Test Acc: 0.8181818181818182
Selected features (CorrelationFS): [30

Fold 0, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.7619047619047619
Selected features (ChiSquareFS): [ 30  35 121 127 129]
Fold 1, VarNum 5, Method CHI2, Model RF, Selected Features: [ 30  35 121 127 129]
Fold 1, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.5652173913043478
Selected features (ChiSquareFS): [ 30  33  35 121 127]
Fold 2, VarNum 5, Method CHI2, Model RF, Selected Features: [ 30  33  35 121 127]
Fold 2, VarNum 5, Method CHI2, Model RF, Train Acc: 1.0, Test Acc: 0.9473684210526315
Selected features (ChiSquareFS): [ 30  35 106 107 127]
Fold 3, VarNum 5, Method CHI2, Model RF, Selected Features: [ 30  35 106 107 127]
Fold 3, VarNum 5, Method CHI2, Model RF, Train Acc: 0.9946808510638298, Test Acc: 0.7391304347826086
Selected features (ChiSquareFS): [ 30  35 106 107 127]
Fold 4, VarNum 5, Method CHI2, Model RF, Selected Features: [ 30  35 106 107 127]
Fold 4, VarNum 5, Method CHI2, Model RF, Train Acc: 0.995260663507109, Test Acc: 0.6666666666

Fold 14, VarNum 5, Method CHI2, Model XGB, Train Acc: 0.9904534606205251, Test Acc: 0.65
Selected features (ChiSquareFS): [30 31 32 33 35]
Fold 15, VarNum 5, Method CHI2, Model XGB, Selected Features: [30 31 32 33 35]
Fold 15, VarNum 5, Method CHI2, Model XGB, Train Acc: 0.9908883826879271, Test Acc: 0.6521739130434783
Selected features (ChiSquareFS): [30 31 32 33 35]
Fold 16, VarNum 5, Method CHI2, Model XGB, Selected Features: [30 31 32 33 35]
Fold 16, VarNum 5, Method CHI2, Model XGB, Train Acc: 0.9913419913419913, Test Acc: 0.6
Selected features (ChiSquareFS): [30 31 32 33 35]
Fold 17, VarNum 5, Method CHI2, Model XGB, Selected Features: [30 31 32 33 35]
Fold 17, VarNum 5, Method CHI2, Model XGB, Train Acc: 0.991701244813278, Test Acc: 0.6
Selected features (ChiSquareFS): [30 31 32 33 35]
Fold 18, VarNum 5, Method CHI2, Model XGB, Selected Features: [30 31 32 33 35]
Fold 18, VarNum 5, Method CHI2, Model XGB, Train Acc: 0.9920318725099602, Test Acc: 0.5454545454545454
Selected featu

...

Fold 12, VarNum 15, Method XGB, Model RF, Train Acc: 1.0, Test Acc: 0.5454545454545454
[15:27:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [107  78  84  45  30  21  50  63  62 131  72  76  69  99  89]
Feature importances: [0.05228566 0.02599358 0.02345364 0.02313786 0.02221133 0.02203631
 0.02164648 0.01945004 0.01928781 0.01914054 0.01795315 0.01788961
 0.0161291  0.01594377 0.01564425]
Fold 13, VarNum 15, Method XGB, Model RF, Selected Features: [107  78  84  45  30  21  50  63  62 131  72  76  69  99  89]
Fold 13, VarNum 15, Method XGB, Model RF, Train Acc: 1.0, Test Acc: 0.7727272727272727
[15:27:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB):

Selected features (XGB): [ 59  25  46   0  61 106  99 134  30  97 113  32 118 133  89]
Feature importances: [0.04915252 0.04541609 0.03848644 0.03071052 0.03060437 0.02968394
 0.02835124 0.02730615 0.02658134 0.02494756 0.02389958 0.0223331
 0.02197508 0.02195759 0.02179354]
Fold 3, VarNum 15, Method XGB, Model XGB, Selected Features: [ 59  25  46   0  61 106  99 134  30  97 113  32 118 133  89]
Fold 3, VarNum 15, Method XGB, Model XGB, Train Acc: 1.0, Test Acc: 0.6956521739130435
[15:27:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [ 91 106  11 134  25  32 107 114  64  35  99 125  72 115  97]
Feature importances: [0.04838402 0.04720221 0.03289761 0.03034894 0.02912888 0.02571377
 0.02553768 0.02345947 0.02330734 0.02305957 0.02195241 0.01955295
 0.01902484 0.01853873 0.01843479]
Fold 4, VarNum 15, Method XGB, Model

Selected features (XGB): [106  93 119  30  45  29  85  33   3 105  40 136 131  55   9]
Feature importances: [0.04404142 0.031406   0.02888564 0.02539387 0.02282043 0.02049493
 0.02032613 0.01902149 0.01853412 0.01766204 0.01700879 0.01694755
 0.01654983 0.015752   0.01557628]
Fold 15, VarNum 15, Method XGB, Model XGB, Selected Features: [106  93 119  30  45  29  85  33   3 105  40 136 131  55   9]
Fold 15, VarNum 15, Method XGB, Model XGB, Train Acc: 1.0, Test Acc: 0.6521739130434783
[15:27:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [ 28  30 106  85  22  93  40 114  63 107  35  94  23  45  42]
Feature importances: [0.04591639 0.02453089 0.02236762 0.02220122 0.02107031 0.01834612
 0.01832638 0.01660874 0.0163494  0.01607329 0.01531808 0.01473181
 0.01466983 0.01447087 0.01388538]
Fold 16, VarNum 15, Method XGB, M

Selected features (XGB): [107  99  17  21  45  64  95 134 106  92 116 130 127  74  32]
Feature importances: [0.05859797 0.05590509 0.05015673 0.02674374 0.02638231 0.02633443
 0.02538574 0.02404596 0.02323602 0.0198393  0.01787091 0.01744338
 0.01704738 0.01699063 0.01666635]
Fold 6, VarNum 15, Method XGB, Model LDA, Selected Features: [107  99  17  21  45  64  95 134 106  92 116 130 127  74  32]
Fold 6, VarNum 15, Method XGB, Model LDA, Train Acc: 0.7250996015936255, Test Acc: 0.36363636363636365
[15:27:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [107  99  66 130  40 134 127  45  25   1 113  22 118  28  30]
Feature importances: [0.0674936  0.05236558 0.03716106 0.03085598 0.02897901 0.0272389
 0.02550623 0.02285499 0.02245414 0.02209218 0.02101223 0.02079713
 0.01824612 0.01747868 0.01745014]
Fold 7, VarNum 15, M

Selected features (XGB): [ 18  73 106 107  30  94  93   1  77  85  79 134  40 112 133]
Feature importances: [0.05220703 0.03001136 0.02681438 0.0255463  0.02420267 0.02270854
 0.02157388 0.02096329 0.0197053  0.01822887 0.01821537 0.0178174
 0.01676521 0.01570135 0.01401956]
Fold 18, VarNum 15, Method XGB, Model LDA, Selected Features: [ 18  73 106 107  30  94  93   1  77  85  79 134  40 112 133]
Fold 18, VarNum 15, Method XGB, Model LDA, Train Acc: 0.6912350597609562, Test Acc: 0.5454545454545454
[15:27:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimator" } are not used.

Selected features (XGB): [ 40  76  17  85 114  13  30  65  71  26 107 106 133  24 126]
Feature importances: [0.03432887 0.03361512 0.02414164 0.02293952 0.02286243 0.02239063
 0.02224347 0.02035736 0.02010566 0.01846889 0.01824656 0.01770644
 0.01687778 0.01592533 0.01554673]
Fold 19, VarNum 15,

In [23]:
pivot_table_exp

,Model,Feature Selection Method,VarNum,Long F1 mean,Monthly Min Return mean,Negative Month Number mean,Negative Test Return sum,Negative Week Number mean,Short F1 mean,Test Accuracy amin,Test Accuracy mean,Test Accuracy std,Test Accuracy Trend mean,Test Return Mean mean,Test Return Mean std,Test Return Mean Trend mean,Weekly Minimum Return mean,Periods,Top Feature 1,Top Feature 2,Top Feature 3,Top Feature 4,Top Feature 5
0,CART,CHI2,5,0.631976,1.022727,0.363636,2,1.318182,0.349783,0.454545,0.620416,0.109283,-0.007910,3.783183,3.378290,-0.121790,-0.806364,22,STOCH_K,WilliamsR,CCI_20,CCI_14,STOCH_D
1,CART,CHI2,10,0.608907,1.282273,0.409091,3,1.227273,0.373068,0.454545,0.622587,0.092905,-0.005521,3.705000,2.724823,-0.108284,-0.530909,22,STOCH_K,CCI_20,WilliamsR,CCI_14,Return_^N225
2,CART,CHI2,15,0.602030,0.320909,0.545455,4,1.454545,0.385031,0.350000,0.595212,0.125675,-0.010707,1.966819,3.058830,-0.206782,-0.929545,22,STOCH_K,CCI_20,WilliamsR,Return_^N225,STOCH_D
3,CART,CORR,5,0.549474,0.597727,0.500000,3,1.590909,0.436962,0.380952,0.562038,0.098837,-0.003754,2.414091,2.958483,0.015375,-0.964091,22,STOCH_K,CCI_14,CCI_20,WilliamsR,RSI_14
4,CART,CORR,10,0.614191,0.512727,0.409091,2,1.636364,0.381627,0.400000,0.619080,0.102985,-0.003606,2.913183,2.856223,-0.026172,-0.825001,22,STOCH_K,CCI_14,CCI_20,WilliamsR,RSI_14
5,CART,CORR,15,0.562036,0.049545,0.500000,4,1.590909,0.432296,0.263158,0.569462,0.139839,-0.007654,1.886819,3.416658,-0.108554,-1.058637,22,STOCH_K,CCI_14,CCI_20,WilliamsR,RSI_14
6,CART,RF,5,0.579162,1.087271,0.454545,3,1.500000,0.408165,0.450000,0.595541,0.094882,-0.004473,3.299544,2.894591,-0.049199,-0.784545,22,STOCH_K,Return_^AXJO,Return_^AORD,Return_^TWII,CCI_20
7,CART,RF,10,0.568988,0.568637,0.590909,5,1.681818,0.415383,0.263158,0.580203,0.126445,-0.003550,2.488637,3.332970,-0.114789,-1.175909,22,Return_^TWII,Return_^AORD,STOCH_K,Return_^AXJO,Return_^N225
8,CART,RF,15,0.612755,0.824545,0.318182,2,1.500000,0.375904,0.350000,0.603993,0.121130,-0.007179,2.883182,3.365338,-0.170869,-0.774546,22,Return_^TWII,Return_^AORD,STOCH_K,Return_^AXJO,Return_^KS11
9,CART,XGB,5,0.525231,-0.147728,0.727273,6,2.136364,0.426082,0.272727,0.545614,0.133173,-0.001877,1.671362,3.551844,0.031694,-1.501820,22,Return_^AXJO,Return_^AORD,STOCH_K,Return_NVDA,AMD


In [ ]:
yfinance: 0.2.37
talib: 0.4.24
numpy: 1.24.4
pandas: 2.0.3
sklearn: 0.24.0
xgboost: 1.7.5
dateutil: 2.8.1
scipy: 1.10.1